# Automated Deployment

- **Estimated time**: ?? minutes
- **Requirements**:
  - How to create [Heat Templates](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2F#HeatOrchestration.ipynb). In fact we will be using the `iperf.yml` template from that Notebook.
  - How to [create a SSH keypair](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2FJupyterOrchestration.ipynb#Create-a-SSH-key-pair)


Previous webinars have successively shown how to automate the orchestration of an experiment from allocating the resources to building the lab to running the experiment and tearing it all down. We began with using the GUI and kept increasing the degree of automation until reaching Heat Templates. The only thing we have not automated so far was lease allocation and reservation. That will change today.

In this notebook we will show how to use Chameleon’s new Automatic Deployment mechanism to start a reservation and deploy the experiment without human intervention. This reservation in turn will call a heat template to orchestrate a lab and run the experiment. As we have been doing in the previous notebooks, once again we will use the iperf lab setup. Contrary to previous notebooks, this one will be very short as the heavy lifting is automated.

## Outline

The proper way to run this in a when creating a lab for and experiment is a single Bash file. However, we will break it up to show the distinct steps in action. 

1. [Step 1: Install the required PIPs](#Step-1%3A-Install-the-required-PIPs)
1. [Step 2: Get the `iperf.yml`](#Step-2%3A-Get-iperf.yml) we used in the previous installment ([Heat Templates](https://jupyter.chameleoncloud.org/import?source=github&src_path=chameleoncloud/notebooks&file_path=tutorials%2Fgetting-started%2F#HeatOrchestration.ipynb).)
1. [Step 3: Initialize Stack](#Step-3%3A-Initialize-Stack), which also starts the experiment.
1. [Step 4: Retrieve the experiment results](#Step-4%3A-Retrieve-the-Experiment-Results) 
1. [Step 5: Tear the entire thing down](#Step-5%3A-Tear-it-Down), including deleting networks, servers, and releasing the floating IP.

### Variables you'll see/use in this Notebook
This should not be anything new to you by now. Note the `$START_DATE` is set to 5 minutes in the future.

In [ ]:
export LEASE_NAME="$USER-automatetest"
export STACK_NAME="$USER-stack"
export START_DATE=`date -d "+5 min" +'%F %H:%M'`
export END_DATE=`date -d "+1 day" +'%F %H:%M'`

## Network Information
# We will use default values

# The Nodes
export NODE_COUNT=3

export SSHKEY_NAME="${USER}-jupyter"
export SSHKEY_FILE="$HOME/work/.ssh/${USER}-jupyter"

export OS_REGION_NAME='CHI@UC'


## Step 1: Install the required PIPs

We need `python-heatclient` and `python-blazarclient`

In [ ]:
pip install -q git+https://github.com/ChameleonCloud/python-heatclient.git
pip install -q git+https://github.com/ChameleonCloud/python-blazarclient.git

## Step 2: Get `iperf.yml`

In [ ]:
curl -o ~/iperf.yml https://raw.githubusercontent.com/raubvogel/notebooks/master/tutorials/getting-started/Orchestration/iperf.yml

## Step 3: Initialize Stack
We can initialize the heat template by passing arguments using the `--parameter` option. Which parameters you can change? Open the heat template file and look under the `parameters`. For this notebook, that will be `iperf.yml`, which has the following parameters defined (case sensitive):

- key_name:
- reservation_id:
- image_name:
- flavor_name:
- private_network_name:
- ip_subnet:

For now we are sticking to the default values for the sake of brevity and only specifying the `$key_name`.

In [ ]:
openstack stack create -t ~/iperf.yml --initialize --parameter reservation_id=dummy \
    --parameter "key_name=${SSHKEY_NAME}" \
    ${STACK_NAME}

If you want to verify if the stack has been created, you can always run

In [ ]:
openstack stack list

We are looking for a `status = INIT_COMPLETE` as shown in the sample output below

```mtavares@36a090ce8167:~$ openstack stack list
+--------------------------------------+----------------+---------------+----------------------+--------------+
| ID                                   | Stack Name     | Stack Status  | Creation Time        | Updated Time |
+--------------------------------------+----------------+---------------+----------------------+--------------+
| f728c425-ca45-4c7d-ad77-83856f1edb52 | mtavares-stack | INIT_COMPLETE | 2020-07-13T21:11:01Z | None         |
+--------------------------------------+----------------+---------------+----------------------+--------------+
mtavares@36a090ce8167:~$ ```

Create Lease, which will call our Heat Template at `$START_DATE`

In [ ]:
# We need to know the stack ID
STACK_ID=$(openstack stack show "${STACK_NAME}" -c id -f value)

blazar lease-create --start-date "${START_DATE}" --end-date "${END_DATE}" \
  --physical-reservation min=$((NODE_COUNT)),max=$((NODE_COUNT +1 )),on_start=orchestration:${STACK_ID} \
  ${LEASE_NAME}

## Step 4: Retrieve the Experiment Results
Get the results file from `iperf_client`. We are printing the results directly into the notebook, but we could as well have downloaded it and manipulated it into a plot or something else. 

**Fun fact:** since the heat template `iperf.yml` populates the `/etc/hosts` file in `$SERVER_NAME`, we can refer to the two clients by their hostnames.

In [ ]:
SERVER_NAME=$(openstack server list -c Name -f value | grep ext_server)
SERVER_FLOATING_IP=$(openstack server show "${SERVER_NAME}"  -c addresses -f value | cut -d ',' -f 2 | xargs)
NODE_NAME="iperf_client"

# Wait until we can connect to server on port 22.
ssh_status=""
while [ "$ssh_status" != "Up" ]
do
   sleep 30
   ssh_status=$(nc -z "${SERVER_FLOATING_IP}" 22 && echo "Up" || echo "Down")
done
  
#Start the ssh forwarding agent and add your private key
ssh-agent
ssh-add $SSHKEY_FILE

echo
echo " ========> results  from iperf_client:"
echo

# Get the results
ssh -A \
      -o "UserKnownHostsFile=/dev/null" \
      -o "StrictHostKeyChecking=no" \
      -q \
      -i $SSHKEY_FILE \
      cc@${SERVER_FLOATING_IP} \
      "ssh -o \"UserKnownHostsFile=/dev/null\" -o \"StrictHostKeyChecking=no\"  cc@${NODE_NAME} \"cat results\""

## Step 5: Tear it Down
And now delete the entire thing, which includes deleting the servers, network, and releasing the floating IP.

In [ ]:
openstack stack delete --yes ${STACK_NAME}

## How do we run this as a single step?

Create a single bash script and paste the above code in it.

### Further reading

- Chameleon documentation on [Automated Deployment](https://chameleoncloud.readthedocs.io/en/latest/technical/complex.html#automated-deployment)
